# Creating batch to test partial feedback setting

In [48]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [49]:
import sys
sys.path.append('../')

In [50]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [51]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [52]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [53]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

### Creating configurations to save into a csv file

In [54]:
time_horizon = 1000
T = 6
targets = util.gen_tar_with_len(T)
values = tuple((v, v) for v in targets)
print(values)
feedback_prob = [1 for i in range(T)]
observabilities = [1 for i in range(T)]
feedback_type = "mab"
distributions = []
for i in range(10):
    distributions.append(tuple(util.gen_distr(T)))
print(distributions)
#chosen_distr = tuple(util.gen_distr(T))
#print("chosen = ", chosen_distr)

((0.704, 0.704), (0.959, 0.959), (0.734, 0.734), (0.99, 0.99), (0.744, 0.744), (0.997, 0.997))
[(0.313, 0.156, 0.283, 0.121, 0.119, 0.008), (0.079, 0.335, 0.414, 0.086, 0.027, 0.059), (0.49700000000000005, 0.08700000000000001, 0.15100000000000002, 0.06400000000000002, 0.11200000000000002, 0.08900000000000001), (0.302, 0.252, 0.224, 0.014, 0.084, 0.124), (0.2882882882882883, 0.2172172172172172, 0.1001001001001001, 0.23523523523523523, 0.1061061061061061, 0.05305305305305305), (0.2642642642642643, 0.08908908908908908, 0.3053053053053053, 0.019019019019019017, 0.26626626626626626, 0.056056056056056056), (0.128, 0.127, 0.199, 0.101, 0.061, 0.384), (0.255, 0.296, 0.203, 0.056, 0.052, 0.138), (0.48951048951048953, 0.020979020979020983, 0.19880119880119884, 0.1188811188811189, 0.10689310689310691, 0.06493506493506494), (0.14214214214214213, 0.022022022022022022, 0.001001001001001001, 0.15715715715715717, 0.5405405405405406, 0.13713713713713716)]


In [40]:
g = game.GameWithObservabilities(values, time_horizon, observabilities, feedback_prob, feedback_type = "mab")
g.attackers = [1]
g.defenders = [0]
d = [bm.FB(g, 0)]
prof = []
#for distr in distributions:
#    prof.append(attackers.StochasticAttacker(g, 1, 1, *distr))
#prof.append(attackers.StochasticAttacker(g, 1, 1, *chosen_distr))
prof.append(attackers.StackelbergAttacker(g, 1, 1))
#for i in range(10):
#    prof.append(attackers.SUQR(g,1))
chosen_a_prof = (attackers.BayesianUnknownStochasticAttacker(g, 1, 1))
#chosen_a_prof = attackers.SUQR(g,1)
a = [deepcopy(chosen_a_prof)]
prof.append(chosen_a_prof)
#a = [attackers.StackelbergAttacker(g, 1, 1)]
shuffle(prof)
#print("Real attacker is: ", a[0])
g.set_players(d, a, prof)

In [41]:
def print_header(targets, profiles):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "," + ",".join(["Obs" + str(x) for x in range(len(targets))]) + "," + ",".join(["Feed_prob" + str(x) for x in range(len(targets))]) + ",Feed_type" + "\n" 

In [42]:
def print_row(targets, time_horizon, d, a, p, obs, fprob, ftype):
    return ",".join([str(i) for i in ([d+"_vs_"+a] + [time_horizon] + targets + [d] + [a] + [x for x in p] + [o for o in obs] + [fp for fp in fprob] + [ftype]) ])+"\n"

In [43]:
(print_header(targets, prof))

'Name,T,0,1,2,3,4,5,Defender,Attacker,Profile,Profile,Obs0,Obs1,Obs2,Obs3,Obs4,Obs5,Feed_prob0,Feed_prob1,Feed_prob2,Feed_prob3,Feed_prob4,Feed_prob5,Feed_type\n'

In [44]:
defender =["FB1"]

In [45]:
profile_strings = []
for p in prof:
    profile_strings.append(util.print_adv(p))

In [46]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/Bayesian vs FB"
batch_name = "bd_vs_sto_2"
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Generating the csv and checking everything is fine

In [47]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, prof))
    f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings,observabilities,feedback_prob,feedback_type))

In [32]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,5,Defender,Attacker,...,Obs3,Obs4,Obs5,Feed_prob0,Feed_prob1,Feed_prob2,Feed_prob3,Feed_prob4,Feed_prob5,Feed_type
0,FB1_vs_sto1-0.042-0.196-0.596-0.041-0.092-0.033,1000,0.966,0.87,0.756,0.76,0.956,0.763,FB1,sto1-0.042-0.196-0.596-0.041-0.092-0.033,...,1,1,1,1,1,1,1,1,1,mab
